In [1]:
from user_ops import ft_pool
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import math

import os
import sys
sys.path.append(os.path.join('..', '..', 'keras_frac'))
from fractional_maxpooling import FractionalPooling2D

%matplotlib inline

/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/

In [2]:
class max_pool_net:
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        self.model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
    
    def train(self, batch_size, epochs, datagen=None, train_data=None, callbacks=None):
        if datagen is None and train_data is None:
            print('neither data or generator was passed')
        elif datagen is None and train_data is not None:
            print('training on array data, network type:', type(self).__name__)
            self.history = self.model.fit(x = train_data[0], y = train_data[1], validation_data=train_data[2:], batch_size=batch_size, epochs=epochs, callbacks=callbacks)
        else:
            print('training on datagen data, network type:', type(self).__name__)
            self.history = self.model.fit_generator(datagen, validation_data=train_data[2:], batch_size=batch_size, epochs=epochs, callbacks=callbacks)
        self.weights = self.model.get_weights()
    
    def restart_session(self):
        keras.backend.clear_session()
        
    def get_history(self):
        return self.history
    
    def get_weights(self):
        return self.weights
    
    def load_weights(self):
        self.model.load_weights(self.weights)

        
class frac_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        ratio = 1.35 # 1.33 blocks=3
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
    
    
class ft_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        strides=(math.sqrt(2), math.sqrt(2))
        pool_size=(math.sqrt(2)*2, math.sqrt(2)*2)
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same',))
        self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
        
class avg_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        pool_size=2
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same',))
        self.model.add(keras.layers.AveragePooling2D(pool_size=(pool_size, pool_size), strides=(2,2), padding='same'))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same'))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
            self.model.add(keras.layers.AveragePooling2D(pool_size=(pool_size, pool_size), strides=(2,2), padding='same'))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same'))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())

In [3]:
EPOCHS = 10
BATCH_SIZE = 100
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
ft_net = ft_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
ft_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
ft_net.restart_session()

avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
avg_net.restart_session()

max_net = max_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
max_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
max_net.restart_session()

frac_net = frac_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
frac_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
frac_net.restart_session()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (100, 32, 32, 2)          56        
_________________________________________________________________
lambda_1 (Lambda)            (100, 23, 23, 2)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (100, 23, 23, 2)          38        
_________________________________________________________________
lambda_2 (Lambda)            (100, 16, 16, 2)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (100, 16, 16, 4)          76        
_________________________________________________________________
lambda_3 (Lambda)            (100, 11, 11, 4)          0         
________________________________________________

In [ ]:
# baseline best 2x max pool = 53.19%

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# petruv pristup

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# muj pristup

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 5x5

In [ ]:
ft_net = ft_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
ft_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
ft_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 2x2

In [ ]:
avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
avg_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 3x3

In [ ]:
avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
avg_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 4x4

In [ ]:
avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
avg_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 6x6

In [ ]:
avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
avg_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()
# 7x7

In [ ]:
ft_net = ft_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
ft_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
ft_net.restart_session()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(max_net.get_history().history['val_accuracy'], 'r')
plt.plot(frac_net.get_history().history['val_accuracy'], 'g')
plt.plot(ft_net.get_history().history['val_accuracy'], 'b')
plt.plot(avg_net.get_history().history['val_accuracy'], 'c')
plt.show()